# Algorithmically Assign Abx To CSNs based on Model Predictions

In [74]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pulp import *
import os, glob

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/conorcorbin/.config/gcloud/application_default_credentials.json' 
os.environ['GCLOUD_PROJECT'] = 'som-nero-phi-jonc101' 
%load_ext google.cloud.bigquery

from google.cloud import bigquery
client=bigquery.Client()

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


/Users/conorcorbin/opt/anaconda3/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


### Load Predictions Data For Each Abx Option

Predictions of coverage for each antibiotic selection are stored in csv files using the following directory schema.  This function reads in predictions for each classifier so that each row is CSN and has an estimated probability of coverage for each antibiotic selection.  We implement a test function that compute the AUROC of each model after the predictions have been read in and cross checks it with the AUROC that was computed and stored in an `auroc.txt` text file during the model training procedure

In [81]:
import pdb
from sklearn.metrics import roc_auc_score

def test_load_predictions(df):
    """
    Reads in output of df, computes AUROC for each classifier and asserts that it equals the AUROC
    listed in the auroc.txt file associate with the classfier's directory
    """
    base_path="/Users/conorcorbin/repos/er_infection/results/ast_models/testing/{abx}"
    abx_options = ["Vancomycin",
               "Ampicillin",
               "Cefazolin",
               "Ceftriaxone",
               "Cefepime",
               "Zosyn",
               "Ciprofloxacin",
               "Meropenem",
               "Vancomycin_Meropenem",
               "Vancomycin_Zosyn",
               "Vancomycin_Cefepime",
               "Vancomycin_Ceftriaxone"
               ]
    for abx in abx_options:
        path = base_path.format(abx=abx)
        f_auroc = os.path.join(path, 'auroc.txt')
        with open(f_auroc, 'r') as f:
            auroc = round(float(f.read()), 3)
        
        computed_auroc = round(roc_auc_score(df['%s_label' % abx], df['%s_predictions' % abx]), 3)
        
        assert auroc == computed_auroc
        print("%s_auroc: %s"% (abx, str(auroc)))

def load_predictions():
    """Helper function that loads predictions from AST classifiers for test set data"""
    
    base_path="/Users/conorcorbin/repos/er_infection/results/ast_models/testing/{abx}"
    abx_options = ["Vancomycin",
                   "Ampicillin",
                   "Cefazolin",
                   "Ceftriaxone",
                   "Cefepime",
                   "Zosyn",
                   "Ciprofloxacin",
                   "Meropenem",
                   "Vancomycin_Meropenem",
                   "Vancomycin_Zosyn",
                   "Vancomycin_Cefepime",
                   "Vancomycin_Ceftriaxone"
                   ]
    df = pd.DataFrame()
    for i, abx in enumerate(abx_options):
        path = base_path.format(abx=abx)
        f_path = glob.glob(os.path.join(path, '*predictions.csv'))[0]
        if i == 0:
            df = pd.read_csv(f_path)
            df = df[['anon_id', 'pat_enc_csn_id_coded', 'label', 'predictions']]
            df = df.rename(columns={'label' : '%s_label' % abx,
                                    'predictions' : '%s_predictions' % abx})
        else:
            df_preds = pd.read_csv(f_path)
            df_preds = df_preds[['anon_id', 'pat_enc_csn_id_coded', 'label', 'predictions']]
            df_preds = df_preds.rename(columns={'label' : '%s_label' % abx,
                                                'predictions' : '%s_predictions' % abx})
            df = df.merge(df_preds, how='left', on=['anon_id', 'pat_enc_csn_id_coded'])
    
    return df
    
df = load_predictions()
test_load_predictions(df)

Vancomycin_auroc: 0.726
Ampicillin_auroc: 0.63
Cefazolin_auroc: 0.672
Ceftriaxone_auroc: 0.692
Cefepime_auroc: 0.62
Zosyn_auroc: 0.664
Ciprofloxacin_auroc: 0.603
Meropenem_auroc: 0.672
Vancomycin_Meropenem_auroc: 0.68
Vancomycin_Zosyn_auroc: 0.687
Vancomycin_Cefepime_auroc: 0.696
Vancomycin_Ceftriaxone_auroc: 0.681


In [20]:
print(len(df))
df.head()

1320


,anon_id,pat_enc_csn_id_coded,Vancomycin_label,Vancomycin_predictions,Ampicillin_label,Ampicillin_predictions,Cefazolin_label,Cefazolin_predictions,Ceftriaxone_label,Ceftriaxone_predictions,...,Meropenem_label,Meropenem_predictions,Vancomycin_Meropenem_label,Vancomycin_Meropenem_predictions,Vancomycin_Zosyn_label,Vancomycin_Zosyn_predictions,Vancomycin_Cefepime_label,Vancomycin_Cefepime_predictions,Vancomycin_Ceftriaxone_label,Vancomycin_Ceftriaxone_predictions
0,JC2a03b24,131260812263,0,0.112854,0,0.417290,0,0.663921,1,0.805743,...,1,0.894460,1,0.991629,1,0.961924,1,0.970687,1,0.879299
1,JCe45a3c,131260883970,1,0.332251,1,0.552269,0,0.672755,0,0.764400,...,0,0.865498,1,0.990867,1,0.963683,1,0.966839,1,0.902455
2,JCd235bb,131261001599,0,0.189070,0,0.428636,1,0.719966,1,0.697382,...,1,0.795243,1,0.993864,1,0.949803,1,0.961116,1,0.897402
3,JCd29af0,131261001696,0,0.368145,0,0.346527,0,0.426055,0,0.470948,...,1,0.531085,1,0.951037,1,0.919925,1,0.935975,0,0.846844
4,JCd356bf,131261014293,1,0.384966,1,0.491907,1,0.612188,1,0.661073,...,1,0.845030,1,0.954875,1,0.966087,1,0.921489,1,0.800497


### Get clinician prescribing patterns
This SQL query gathers all abx medications ordered within the first 24 hours of admission that were administered to the patient in long format ( one row per administered med_description ) and then joins to our labels table so that we can cross check whether the administered antibiotic was sufficient to cover the patient. 

In [82]:
query = """
SELECT
    om.anon_id, om.pat_enc_csn_id_coded, om.order_med_id_coded, l.index_time, om.med_description,
    l.Ampicillin, l.Ciprofloxacin, l.Cefazolin, l.Ceftriaxone, l.Cefepime, l.Zosyn, l.Vancomycin,
    l.Meropenem, l.Vancomycin_Meropenem, l.Vancomycin_Zosyn, l.Vancomycin_Cefepime, l.Vancomycin_Ceftriaxone
FROM
    `mining-clinical-decisions.abx.abx_orders_given_and_stopped` om
INNER JOIN 
    `mining-clinical-decisions.abx.final_ast_labels` l
USING
    (pat_enc_csn_id_coded)
WHERE
    om.was_given = 1
ORDER BY 
    om.anon_id, om.pat_enc_csn_id_coded, om.order_time
"""
query_job = client.query(query)
df_abx = query_job.result().to_dataframe()
df_abx.head()

,anon_id,pat_enc_csn_id_coded,order_med_id_coded,index_time,med_description,Ampicillin,Ciprofloxacin,Cefazolin,Ceftriaxone,Cefepime,Zosyn,Vancomycin,Meropenem,Vancomycin_Meropenem,Vancomycin_Zosyn,Vancomycin_Cefepime,Vancomycin_Ceftriaxone
0,JC29f92d0,131265245074,603371616,2019-02-20 10:49:00+00:00,CEFTRIAXONE 1 GRAM/100 ML NS MINIBAG PLUS,0,0,0,0,0,1,0,1,1,1,0,0
1,JC29f92d0,131265245074,603374106,2019-02-20 10:49:00+00:00,PIPERACILLIN-TAZOBACTAM-DEXTRS 3.375 GRAM/50 M...,0,0,0,0,0,1,0,1,1,1,0,0
2,JC29f92d0,131265245074,603418477,2019-02-20 10:49:00+00:00,ERTAPENEM 1 GRAM/50 ML NS MINIBAG PLUS,0,0,0,0,0,1,0,1,1,1,0,0
3,JC29f9afd,131114292966,476869102,2015-09-03 14:08:00+00:00,VANCOMYCIN IN D5W 1 GRAM/200 ML IV PGBK,1,0,1,1,1,1,1,1,1,1,1,1
4,JC29f9afd,131114292966,476869103,2015-09-03 14:08:00+00:00,PIPERACILLIN-TAZOBACTAM-DEXTRS 4.5 GRAM/100 ML...,1,0,1,1,1,1,1,1,1,1,1,1


### Aggregate antibiotic orders 
Here we aggregate the antibiotic orders so that one row in the desulting dataframe corresponds to a unique CSN. We do this by
1. Grouping by the CSN
2. Grabbing the first word (antibiotic name) from the med description
3. Aggregating the `med_description` column such that it is a single string with all antibiotics admistered to the patient, sorted in alphabetical order and separated by spaces. 
4. Only keep CSNs where the set of administered antibiotics is equal to one of the antbiotic selections we've trained classifiers for. 

In [99]:
# Useful dictionaries to map corresponding. strings for the sameantibiotic selections
abx_map = {'Ceftriaxone' : "CEFTRIAXONE",
           'Vancomycin_Zosyn' : "PIPERACILLIN-TAZOBACTAM VANCOMYCIN",
           'Zosyn' : "PIPERACILLIN-TAZOBACTAM",
           'Vancomycin_Ceftriaxone' : "CEFTRIAXONE VANCOMYCIN",
           'Vancomycin_Cefepime' : "CEFEPIME VANCOMYCIN",
           'Cefepime' : "CEFEPIME",
           'Vancomycin' :  "VANCOMYCIN",
           'Meropenem' : "MEROPENEM",
           'Vancomycin_Meropenem' : "MEROPENEM VANCOMYCIN",
           'Cefazolin' : "CEFAZOLIN",
           'Ciprofloxacin' : "CIPROFLOXACIN",
           'Ampicillin' : 'AMPICILLIN'
          }
abx_map_inverse = {abx_map[key] : key for key in abx_map}

# Lambda that aggregate Antibiotic orders after we've grouped by CSN
concat_abx = lambda x : ' '.join(np.unique(sorted([a for a in x])))

# 
df_drugs = (df_abx
    .assign(med_description=lambda x: [a.split(' ')[0] for a in x.med_description]) # Only Take first word (abx)
    .assign(med_description=lambda x: [(a.replace('PIPERACILLIN-TAZOBACTAM-DEXTRS','PIPERACILLIN-TAZOBACTAM')
                                        .replace('VANCOMYCIN-WATER', 'VANCOMYCIN'))
                                       for a in x.med_description])
    .assign(year=lambda x: x.index_time.dt.year) # get year of each CSN - used to filter later on
    .groupby('pat_enc_csn_id_coded')
    .agg({'med_description' : concat_abx,
          'year' : 'first',
          'Ampicillin' : 'first',
          'Ciprofloxacin' : 'first',
          'Cefazolin' : 'first',
          'Ceftriaxone' : 'first',
          'Cefepime' : 'first',
          'Zosyn' : 'first',
          'Vancomycin' : 'first',
          'Meropenem' : 'first',
          'Vancomycin_Ceftriaxone' : 'first',
          'Vancomycin_Cefepime' : 'first',
          'Vancomycin_Zosyn' : 'first',
          'Vancomycin_Meropenem' : 'first'})
    .reset_index()
    # Only look at test set data and CSNs where allowed antibiotic selection was administered
    .query("year == 2019 and med_description in @abx_map_inverse", engine='python') 
)

# Roughly 700 of the 1300 original CSNs in the test set
print(len(df_drugs))
df_drugs.head()

697


,pat_enc_csn_id_coded,med_description,year,Ampicillin,Ciprofloxacin,Cefazolin,Ceftriaxone,Cefepime,Zosyn,Vancomycin,Meropenem,Vancomycin_Ceftriaxone,Vancomycin_Cefepime,Vancomycin_Zosyn,Vancomycin_Meropenem
6847,131260812263,CEFTRIAXONE,2019,0,1,0,1,1,1,0,1,1,1,1,1
6858,131261001599,PIPERACILLIN-TAZOBACTAM,2019,0,1,1,1,1,1,0,1,1,1,1,1
6861,131261014293,CEFTRIAXONE,2019,1,1,1,1,1,1,1,1,1,1,1,1
6878,131261155365,CEFTRIAXONE,2019,1,1,1,1,1,1,0,1,1,1,1,1
6879,131261167965,MEROPENEM VANCOMYCIN,2019,1,1,1,1,1,1,1,1,1,1,1,1


### Merge this dataframe to predictions dataframe
After this step we should have a dataframe that has one row per CSN, each row should have the antibiotic selection actually administered to the patient, along with the predicted probability of said antibiotic selection covering the patient, and the ground truth as to whether it did. 

In [100]:
# Merge df_total to df on pat_enc_csn_id_coded
df_new = (df
    .merge(df_drugs, how='inner', on='pat_enc_csn_id_coded')
)

# Sanity check - make sure %abx_label columns are equal to %abx columns
for abx in abx_options:
    for i in range(len(df_new)):
        assert df_new[abx].values[i] == df_new['%s_label' % abx].values[i]
        
# Sanity check 2: compute AUROC of this subset of patients and compare to AUROC on full test set
for abx in abx_options:
    computed_auroc = roc_auc_score(df_new['%s_label' % abx], df_new['%s_predictions' % abx])
    f_auroc = os.path.join(base_path.format(abx=abx), 'auroc.txt')
    with open(f_auroc, 'r') as f:
        auroc = float(f.read())
    print("{}: Full test set AUROC:{:.3f} Subset AUROC:{:.3f}".format(abx, auroc, computed_auroc))

Vancomycin: Full test set AUROC:0.726 Subset AUROC:0.712
Ampicillin: Full test set AUROC:0.630 Subset AUROC:0.603
Cefazolin: Full test set AUROC:0.672 Subset AUROC:0.678
Ceftriaxone: Full test set AUROC:0.692 Subset AUROC:0.689
Cefepime: Full test set AUROC:0.620 Subset AUROC:0.624
Zosyn: Full test set AUROC:0.664 Subset AUROC:0.619
Ciprofloxacin: Full test set AUROC:0.603 Subset AUROC:0.608
Meropenem: Full test set AUROC:0.672 Subset AUROC:0.667
Vancomycin_Meropenem: Full test set AUROC:0.680 Subset AUROC:0.739
Vancomycin_Zosyn: Full test set AUROC:0.687 Subset AUROC:0.661
Vancomycin_Cefepime: Full test set AUROC:0.696 Subset AUROC:0.764
Vancomycin_Ceftriaxone: Full test set AUROC:0.681 Subset AUROC:0.700


### Create Binary Integer Programming Problem Formulation and Solve
Here we specificy the problem formulation of the optimization process we wish to solve. The goal is to maximize the probability of covering the set of patients in the test set with the available antibiotic selections subject to the constraints that we assign each antibiotic selection a prespecified number of times, and that we only assign one antibiotic selection to each patient CSN. 

More technically, Let $N$ be the number of patient CSNs in our test set who were administered one of the 12 abx selections by clinicians, and let $K$ be the number of possible antibiotic selections.  Let $A$ be a matrix in $\mathbb{R}^{N\times K}$ such that $a_{ij}$ is 1 if antibiotic selection $j$ is selected for patient CSN $i$ and 0 otherwise. Let $\Phi$ be a matrix in $\mathbb{R}^{N \times K}$ such that $\phi_{ij}$ is the predicted probability that antibiotic $j$ will cover patient CSN $i$.  Let $C$ be a vector in $\mathbb{R}^K$ such that $c_j$ specifies the budget for anitbiotic selection $j$ - that is the number of times we are allowed to select antibiotic $j$ across our $N$ patient CSNs. Our problem formulation is as follows. 

$$  \underset{A}{\text{maximize}} \sum_{i=1}^{N} \sum_{j=1}^K \phi_{ij} a_{ij} $$

Subject to the following constraints:

$$ \sum_{j=1}^{K} a_{ij} = 1 \quad i = 1, ..., N $$

$$ \sum_{i=1}^{N} a_{ij} = c_j \quad j = 1, ...,  K $$

In the following code, we implenent and solve this optimization process using the pulp python package. 

In [119]:
abx_model = LpProblem("Antibiotics", LpMaximize)

# Create binary indicators for whether treatment is used
drug_inds = {}
for abx in abx_options:
    drug_inds[abx] = [LpVariable('%s_%d' % (abx, i), lowBound=0, upBound=1, cat='Binary')
                      for i in range(len(df_new))]

# Add objective function to model
per_csn_sum = []
for i in range(len(df_new)):
    _sum = 0
    for abx in abx_options:
        _sum += drug_inds[abx][i] * df_new['%s_predictions' % abx].values[i]
    per_csn_sum.append(_sum)
    
abx_model += lpSum(per_csn_sum)

# Add one selection constraint
for i in range(len(df_new)):
    selections = []
    for abx in abx_options:
        selections.append(drug_inds[abx][i])
    abx_model += lpSum(selections) == 1

# Add max assignment constraints
abx_assignment_constraints = {"Vancomycin" : 13,
                              "Ampicillin" : 0,
                              "Cefazolin" : 8,
                              "Ceftriaxone" : 367,
                              "Cefepime" : 14,
                              "Zosyn" : 102,
                              "Ciprofloxacin" : 8,
                              "Meropenem" : 9,
                              "Vancomycin_Meropenem" : 9,
                              "Vancomycin_Zosyn" :  113,
                              "Vancomycin_Cefepime" : 23,
                              "Vancomycin_Ceftriaxone" : 31
                             }

for drug in drug_inds:
    abx_model += lpSum([drug_inds[drug][i] for i in range(len(df_new))]) == abx_assignment_constraints[drug]

# Solve model
abx_model.solve()
print("Status:", LpStatus[abx_model.status])

# Save selected antibiotic to df_new
abx_decisions = []
for i in range(len(df_new)):
    abx_decision = None
    for abx in abx_options:
        if drug_inds[abx][i].value() == 1:
            abx_decision = abx_map[abx]
    assert abx_decision is not None
    abx_decisions.append(abx_decision)
df_new['IP_med_description'] = abx_decisions


Status: Optimal


### Compare Performance to Clinician Performance
1. Write a function that takes in antibiotic selection and outputs a 1 if that selection covered the patient.  Simple to do, but annoying because of different ways we've named antibiotic selections.
2. Compute fraction of time each patient CSN was covered by the antibiotic selection. 

In [120]:
# Ugly helper function that just does some string mapping
def compute_was_covered(x, decision_column='med_description'):
    """
    Given med description, find appropriate label column and return whether patient was covered during CSN
    Returns "Not in abx options" if abx regimen isn't in our set of 12 options - useful for filtering later
    """
    if decision_column == 'med_description':
        med_description = x.med_description
    elif decision_column == 'random_med_description':
        med_description = x.random_med_description
    elif decision_column == 'IP_med_description':
        med_description = x.IP_med_description
        
    if med_description == "CEFTRIAXONE":
        return x.Ceftriaxone
    elif med_description == "PIPERACILLIN-TAZOBACTAM VANCOMYCIN":
        return x.Vancomycin_Zosyn
    elif med_description == "PIPERACILLIN-TAZOBACTAM":
        return x.Zosyn
    elif med_description == "CEFTRIAXONE VANCOMYCIN":
        return x.Vancomycin_Ceftriaxone
    elif med_description == "CEFEPIME VANCOMYCIN":
        return x.Vancomycin_Cefepime
    elif med_description == "CEFEPIME":
        return x.Cefepime
    elif med_description == "VANCOMYCIN":
        return x.Vancomycin
    elif med_description == "MEROPENEM":
        return x.Meropenem
    elif med_description == "MEROPENEM VANCOMYCIN":
        return x.Vancomycin_Meropenem
    elif med_description == "CEFAZOLIN":
        return x.Cefazolin
    elif med_description == "CIPROFLOXACIN":
        return x.Ciprofloxacin
    elif med_description == "AMPICILLIN":
        return x.Ampicillin
    else:
        return "Not in abx options"

    
# Create flag for whether clinicians covered the patient during the csn, whether a random assignemnt covered patient
# CSN, and whether optimized assignment covered the patient CSN.

df_new = (df_new
    .assign(random_med_description=lambda x: np.random.choice(x.med_description, size=len(x.med_description), replace=False))
)
df_new = (df_new
    #.sample(frac=1.0, replace=True) # bootstrap each iteration
    .assign(was_covered_dr=df_new.apply(lambda x: compute_was_covered(x), axis=1))
    .assign(was_covered_random=df_new.apply(lambda x: compute_was_covered(x, 
                                                                          decision_column='random_med_description'),
                                                                          axis=1))
    .assign(was_covered_IP=df_new.apply(lambda x: compute_was_covered(x, 
                                                                      decision_column='IP_med_description'),
                                                                      axis=1))
)

clin_covered_rate = df_new['was_covered_dr'].sum() / len(df_new)
random_covered_rate = df_new['was_covered_random'].sum() / len(df_new)
ip_covered_rate = df_new['was_covered_IP'].sum() / len(df_new)

print(clin_covered_rate)
print(random_covered_rate)
print(ip_covered_rate)

df_new_random = (df_new
        .groupby('random_med_description')
        .agg(num_distinct_csns=('pat_enc_csn_id_coded', 'count'),
             num_times_covered_random=('was_covered_random', 'sum'))
        .reset_index()
        .assign(random_covered=lambda x: ['{}/{}'.format(c, t) for c, t in zip(x.num_times_covered_random,
                                                                               x.num_distinct_csns)])
        .rename(columns={'random_med_description' : 'med_description'})
)[['med_description', 'random_covered']]
                 
df_new_clinician = (df_new
        .groupby('med_description')
        .agg(num_distinct_csns=('pat_enc_csn_id_coded', 'count'),
             num_times_covered_dr=('was_covered_dr', 'sum'))
        .reset_index()
        .assign(dr_covered=lambda x: ['{}/{}'.format(c, t) for c, t in zip(x.num_times_covered_dr,
                                                                               x.num_distinct_csns)])
)[['med_description', 'dr_covered']]
                    
df_new_ip = (df_new
        .groupby('IP_med_description')
        .agg(num_distinct_csns=('pat_enc_csn_id_coded', 'count'),
             num_times_covered_IP=('was_covered_IP', 'sum'))
        .reset_index()
        .assign(IP_covered=lambda x: ['{}/{}'.format(c, t) for c, t in zip(x.num_times_covered_IP,
                                                                               x.num_distinct_csns)])
        .rename(columns={'IP_med_description' : 'med_description'})
)[['med_description', 'IP_covered']]

df_new_agg = (df_new_random
    .merge(df_new_clinician, how='inner', on='med_description')
    .merge(df_new_ip, how='inner', on='med_description')
)

df_new_agg

0.8393113342898135
0.7747489239598279
0.8464849354375896


,med_description,random_covered,dr_covered,IP_covered
0,CEFAZOLIN,3/8,5/8,8/8
1,CEFEPIME,9/14,11/14,12/14
2,CEFEPIME VANCOMYCIN,22/23,22/23,21/23
3,CEFTRIAXONE,263/367,283/367,287/367
4,CEFTRIAXONE VANCOMYCIN,21/31,30/31,30/31
5,CIPROFLOXACIN,7/8,7/8,7/8
6,MEROPENEM,6/9,7/9,6/9
7,MEROPENEM VANCOMYCIN,8/9,9/9,8/9
8,PIPERACILLIN-TAZOBACTAM,90/102,93/102,94/102
9,PIPERACILLIN-TAZOBACTAM VANCOMYCIN,108/113,109/113,106/113
